In [ ]:
import csv
import time

from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener

from glass.cons.dsn import search_words, tw_key

In [ ]:
k = tw_key(allkeys=None)

words = search_words(group='fire')

In [ ]:
class TwListener(StreamListener):
    
    def __init__(self, start_time, time_limit):
        self.time  = start_time
        self.limit = time_limit
        
        super(TwListener, self).__init__()

    def on_status(self, status):
        if (time.time() - start_time) > self.limit:
            return False
        
        if status.lang == "en":
            # Altering text so that it keeps to one line
            out_txt = "'" + status.text.replace('\n', ' ') + "'"
            
            csvw.writerow([
                status.id,
                status.user.screen_name,
                status.created_at.strftime('%m/%d/%y'),
                status.user.followers_count,
                out_txt
            ])
            
        return True

    def on_error(self, status):
        if status == 420:
            return False

In [ ]:
auth = OAuthHandler(k['CONSUMER_KEY'], k['CONSUMER_SECRET'])

auth.set_access_token(k['TOKEN'], k['SECRET'])

start_time = time.time()

tw_stream = Stream(auth, TwListener(start_time, 20))

# Open file to write data
with open("/home/jasp/mrgis/test_stream.csv", "w", newline='') as csvfile:
    csvw = csv.writer(csvfile, delimiter=' ', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    
    csvw.writerow(["twitter_id", "name", "created_at", "followers_count", 'text'])

    tw_stream.filter(track=words.word.tolist())